In [1]:
# Derived from keras-rl
import opensim as osim
import numpy as np
import sys

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, merge
from keras.optimizers import Adam

import numpy as np

from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

from osim.env import *

from keras.optimizers import RMSprop

import argparse
import math

print("End of importation")

Using TensorFlow backend.


End of importation


In [2]:
print("Arguments...")
#print(args)

args = argparse.Namespace(model='example.h5f', steps=10000, visualize=False)
print(args)
# Load walking environment
env = ArmEnv(args.visualize)
print("Environment generated")



Arguments...
Namespace(model='example.h5f', steps=10000, visualize=False)
Environment generated


In [3]:
nb_actions = env.action_space.shape[0]

# Total number of steps in training
nallsteps = args.steps

# Create networks for DDPG
# Next, we build a very simple model.
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('sigmoid'))
print(actor.summary())

action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = merge([action_input, flattened_observation], mode='concat')
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('linear')(x)
critic = Model(input=[action_input, observation_input], output=x)
print(critic.summary())

# Set up the agent for training
#warning: a too high limit gives errors
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(theta=.15, mu=0., sigma=.2, size=env.noutput)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3,
                  delta_clip=1.)
# agent = ContinuousDQNAgent(nb_actions=env.noutput, V_model=V_model, L_model=L_model, mu_model=mu_model,
#                            memory=memory, nb_steps_warmup=1000, random_process=random_process,
#                            gamma=.99, target_model_update=0.1)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_1 (Flatten)              (None, 14)            0           flatten_input_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 32)            480         flatten_1[0][0]                  
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32)            0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 32)            1056        activation_1[0][0]               
___________________________________________________________________________________________

In [ ]:
print(action_input)
print(observation_input)

In [4]:
nallsteps=10000
#Warning: verbose=1 freezes the notebook
agent.fit(env, nb_steps=nallsteps, visualize=False, verbose=2, nb_max_episode_steps=env.timestep_limit, log_interval=10000)
# After training is done, we save the final weights.
agent.save_weights(args.model, overwrite=True)

Training for 100000 steps ...
   500/100000: episode: 1, duration: 10.582s, episode steps: 500, steps per second: 47, episode reward: -579.349, mean reward: -1.159 [-3.245, -0.133], mean action: 0.322 [-0.645, 1.043], mean observation: -0.531 [-1000.000, 1000.000], loss: 0.959434, mean_absolute_error: 1.119451, mean_q: -1.190162
  1000/100000: episode: 2, duration: 10.345s, episode steps: 500, steps per second: 48, episode reward: -249.679, mean reward: -0.499 [-1.036, -0.179], mean action: 0.374 [-0.433, 1.273], mean observation: -0.498 [-923.802, 897.097], loss: 0.744982, mean_absolute_error: 0.900031, mean_q: -1.652770
  1500/100000: episode: 3, duration: 9.116s, episode steps: 500, steps per second: 55, episode reward: -137.106, mean reward: -0.274 [-1.496, -0.020], mean action: 0.506 [-0.413, 1.420], mean observation: -0.137 [-346.360, 322.495], loss: 0.301973, mean_absolute_error: 0.409812, mean_q: -1.380452
  2000/100000: episode: 4, duration: 12.381s, episode steps: 500, steps 

In [6]:
agent.load_weights(args.model)
# Finally, evaluate our algorithm for 1 episode.
agent.test(env, nb_episodes=5, visualize=True, nb_max_episode_steps=500)

Testing for 5 episodes ...
Episode 1: reward: -91.295, steps: 500
Episode 2: reward: -38.727, steps: 500
Episode 3: reward: -40.374, steps: 500
Episode 4: reward: -198.706, steps: 500
Episode 5: reward: -168.487, steps: 500
